<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#用Class来封装繁琐的字典和tuples" data-toc-modified-id="用Class来封装繁琐的字典和tuples-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>用Class来封装繁琐的字典和tuples</a></span></li><li><span><a href="#利用call方法将class当成hook" data-toc-modified-id="利用call方法将class当成hook-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>利用<strong>call</strong>方法将class当成hook</a></span></li><li><span><a href="#用类方法来生成类的实例" data-toc-modified-id="用类方法来生成类的实例-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>用类方法来生成类的实例</a></span></li><li><span><a href="#用-super().__init__初始化父类" data-toc-modified-id="用-super().__init__初始化父类-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>用 super().__init__初始化父类</a></span></li><li><span><a href="#Min-in-基础类" data-toc-modified-id="Min-in-基础类-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Min-in 基础类</a></span></li><li><span><a href="#不要用私有成员,-用受保护的成员-+-文档标注" data-toc-modified-id="不要用私有成员,-用受保护的成员-+-文档标注-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>不要用私有成员, 用受保护的成员 + 文档标注</a></span></li><li><span><a href="#当基类的成员名字和子类的成员名字可能冲突时,-考虑私有成员" data-toc-modified-id="当基类的成员名字和子类的成员名字可能冲突时,-考虑私有成员-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>当基类的成员名字和子类的成员名字可能冲突时, 考虑私有成员</a></span></li><li><span><a href="#从-collections.abc-继承传统的容器类" data-toc-modified-id="从-collections.abc-继承传统的容器类-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>从 collections.abc 继承传统的容器类</a></span></li></ul></div>

### 用Class来封装繁琐的字典和tuples

In [1]:
#我们用namedtuple模块来抽象 Grade对象
from collections import namedtuple
Grade =  namedtuple('Grade', ('score', 'weight'))

#namedtuple 的几个注意点
##1. 你无法为namedtuple指定默认值
##2. namedtuple的属性仍然可以通过索引和迭代来访问

#因此做好权衡,要么自己定义Grade Class

In [2]:
#下面定义学科类,他的元素是 grade的列表
class Subject(object):
    def __init__(self):
        self._grades = []
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [3]:
#最后定义学生类,他有一个字典,里面储存了每个学科
class Student(object):
    def __init__(self):
        self._subjects = {}
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [ ]:
#最后定义 成绩单类, 储存了每个学生
class Gradebook(object):
    def __init__(self):
        self._students = {}
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [ ]:
##层层封装,这样每个类的属性都是 另一个更小的类的实例的字典/列表

### 利用__call__方法将class当成hook

In [1]:
from collections import defaultdict
current = {'green': 12, 'blue': 13}
increments = [
    ('red', 5), 
    ('blue', 17), 
    ('orange', 9),
]
def log_missing():
    print('Key added')
    return 0

result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After:', dict(result))

Before: {'green': 12, 'blue': 13}
Key added
Key added
After: {'green': 12, 'blue': 30, 'red': 5, 'orange': 9}


In [2]:
#上面的实现不错, 它易于测试和阅读

In [4]:
#如果你想为log_missing增加一些复杂的功能 比如维持一个状态，
#你可以考虑封装成class

class CountMissing(object):
    def __init__(self):
        self.added = 0
    def __call__(self):
        self.added += 1
        return 0
counter = CountMissing()
assert callable(counter)

In [5]:
counter = CountMissing()
#会调用__call__
result = defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
print(counter.added)

2


### 用类方法来生成类的实例

In [6]:
#假设你要实现一个MapReduce

from threading import Thread
#多线程执行函数
def excute(workers):
    #多线程map
    threads = [Thread(target = w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    #将结果reduce到第一个worker
    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result

In [7]:
#下面实现读取数据的接口
import os
#一个抽象的读取数据的基类
class GenericInputData(object):
    def read(self):
        raise NotImplementedError
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

class PathInputData(object):
    def __init__(self, path):
        super().__init__()
        self.path = path
    def read(self):
        return open(self.path).read()
    
    #每一个PathInputData的实例负责读取一个文件
    #用类方法将生成类的实例的代码包含在类里面,使得代码更加紧凑
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

In [9]:
#worker接口
class GenericWorker(object):
    def map(self):
        raise NotImplementedError
    def reduce(self, other):
        raise NotImplementedError
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers
class LineCountWorker(GenericWorker):
    pass

In [11]:
#mapreduce 函数将上面的两个类连接起来
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return excute(workers)

#mapreduce(LineCountWorker, PathInputData, config)

### 用 super().\_\_init\_\_初始化父类

### Min-in 基础类

In [12]:
#Min-in 是一个很小的类,只提供了一些额外的操作来让继承它的子类使用

In [16]:
#ToDictMixin类提供了返回类成员的嵌套字典的方法 
class ToDictMixin(object):
    #返回类本身的字典
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    #中间函数，对类的每个属性和方法进行转换: 类的属性名/方法名 : <ahfduioahgfiowe>
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    #最底层的转换函数, 按照value的不同做不同的处理
    #将实现细节封装到最底层！！
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse_dict(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [17]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left = None, right = None):
        self.value = value
        self.left = left
        self.right = right

In [18]:
tree = BinaryTree(10, left = BinaryTree(7, right = BinaryTree(9)),
                  right = BinaryTree(13, left = BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


In [19]:
#你可以在需要的时候, 子类中覆盖Mix-in基类的方法
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left = None, right = None, parent = None):
        super().__init__(value, left = left, right = right)
        self.parent = parent
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and key == 'parent'):
            #避免死循环！！
            return value.value
        else:
            return super()._traverse(key, value)

In [20]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent = root)
root.left.right = BinaryTreeWithParent(9, parent = root.left)

print(root.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10}, 'right': None, 'parent': None}


In [22]:
#在子类中覆盖了mix-in的方法后, 任何含有子类实例成员的类也会覆盖掉mix-in的方法:
class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent
my_tree = NamedSubTree('foobar', root.left.right)
#OK!
print(my_tree.to_dict())

{'name': 'foobar', 'tree_with_parent': {'value': 9, 'left': None, 'right': None, 'parent': 7}}


In [25]:
#子类可以继承多个mix-in:
#1.子类的__init__需要接受关键字参数赋值
#2. 子类需要有to_dict方法
class JsonMixin(object):
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    def to_json(self):
        return json.dumps(self.to_dict())

In [26]:
class DatacenterRack(ToDictMixin, JsonMixin):
    pass

### 不要用私有成员, 用受保护的成员 + 文档标注
### 当基类的成员名字和子类的成员名字可能冲突时, 考虑私有成员

### 从 collections.abc 继承传统的容器类

In [4]:
#你可以继承现有的容器类
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts
    

In [5]:
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])

In [6]:
foo.frequency()

{'a': 3, 'b': 2, 'c': 1, 'd': 1}

In [9]:
#或者自己实现magic方法，例如你想实现一个支持索引的二叉树类
class BinaryNode(object):
    def __init__(self, value, left = None, right = None):
        self.value = value
        self.left = left
        self.right = right
#为了支持索引,你得自己实现 __getitem__方法
class IndexableNode(BinaryNode):
    def _search(self, count, index):
        #returns (found, counts)
        pass
    def __getitem__(self, index):
        found, _ = self._search(0, index)
        if not found:
            raise IndexError('Index out of range!')
        return found.value

In [12]:
#当然,如果你想要使你的二叉树类更像序列容器,你需要更多的__magic__方法
#你可以继承collections.abc里面的容器类型
from collections.abc import Sequence
#这样你必须实现所有需要的magic method， 否则初始化会失败。